<a href="https://colab.research.google.com/github/MelchorMS/ModeloLogit/blob/main/ModeloLogit_Diabetes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
install.packages("readr")
install.packages("dplyr")
install.packages("tidyr")
install.packages("pscl")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



# **1. Modelo Logit: Análisis de Factores de Riesgo para la Diabetes**
##1.1 Introducción

Este estudio emplea un modelo de regresión logística binomial con un conjunto de datos individuales para analizar la relación entre características clínicas, metabólicas y demográficas con la probabilidad de desarrollar diabetes mellitus en pacientes adultos. Nuestra variable de respuesta está definida por la presencia o ausencia de diabetes (1 = diabético, 0 = no diabético), mientras que las variables predictoras incluyen:

*   Edad: Edad del paciente (años)
*   Glucosa: Concentración de glucosa en plasma
*   Insulina: Insulina sérica de 2 horas (mu U/ml)
*   Índice de Masa Corporal (BMC) (kg/m²)
*   DiabetesPedigreeFunction (Función que asigna la probabilidad de padecer diabetes a partir de la historia familiar)

La importancia de este analisis radica en el gran uso que se le pueden dar a estos datos clinicos como herramientas para la creacion de politicas de prevencion dentro de la salud publica, apartir de identificar los factores de riesgo clave asociados a la diabetes.
Por lo que podriamos obtener un analisis cualitativo que dentro de la economia de la salud, nos permita optimizar el uso de recursos limitados para disminuir esta problematica dentro de la poblaciòn en general.

**METODOLOGIA:**

La metodologia a usar sera un modelo de regresiòn logistica binomial, la cual nos permitira analizar de manera robusta la relacion entre las variables predictoras junto con un resultado binario (en este caso, presencia o ausencia de diabetes).
La elecciòn de este modelo se debe ya que al contar con datos individuales, no se puede recurrir al metodo clasico de MCO por lo que se recurrira a procedimientos de cálculo no lineales como el método de máxima verosimilitud.

In [ ]:
library("readr")
df <- read_csv("/content/diabetes.csv")

Rows: 768 Columns: 6
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
dbl (6): diabetes, Age, Glucose, Insulin, BMI, DiabetesPedigreeFunction

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [ ]:
# Cargar paquetes necesarios
library(dplyr)
library(tidyr)

# Variables a imputar
vars_a_imputar <- c("Glucose", "Insulin", "BMI")

# Reemplazar ceros y NA por la mediana dentro de las variables independientes
df <- df %>%
  mutate(across(all_of(vars_a_imputar), ~ ifelse(. == 0, NA, .))) %>%  # Convertir ceros en NA
  mutate(across(all_of(vars_a_imputar), ~ replace_na(., median(., na.rm = TRUE))))  # Imputar con mediana


In [ ]:
#Verificamos si quedan 0 dentro de nuestras variables reemplazadas
vars_a_imputar <- c("Glucose", "Insulin", "BMI")

sapply(df[vars_a_imputar], function(x) sum(x == 0))

Glucose Insulin     BMI 
      0       0       0

In [ ]:
head(df)
summary(df)

diabetes,Age,Glucose,Insulin,BMI,DiabetesPedigreeFunction
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,50,148,125,33.6,0.627
0,31,85,125,26.6,0.351
1,32,183,125,23.3,0.672
0,21,89,94,28.1,0.167
1,33,137,168,43.1,2.288
0,30,116,125,25.6,0.201


    diabetes          Age           Glucose          Insulin     
 Min.   :0.000   Min.   :21.00   Min.   : 44.00   Min.   : 14.0  
 1st Qu.:0.000   1st Qu.:24.00   1st Qu.: 99.75   1st Qu.:121.5  
 Median :0.000   Median :29.00   Median :117.00   Median :125.0  
 Mean   :0.349   Mean   :33.24   Mean   :121.66   Mean   :140.7  
 3rd Qu.:1.000   3rd Qu.:41.00   3rd Qu.:140.25   3rd Qu.:127.2  
 Max.   :1.000   Max.   :81.00   Max.   :199.00   Max.   :846.0  
                 NA's   :1                                       
      BMI        DiabetesPedigreeFunction
 Min.   :18.20   Min.   :0.0780          
 1st Qu.:27.50   1st Qu.:0.2437          
 Median :32.30   Median :0.3725          
 Mean   :32.46   Mean   :0.4719          
 3rd Qu.:36.60   3rd Qu.:0.6262          
 Max.   :67.10   Max.   :2.4200          
                                         

## 1.2 Ecuacion del modelo:
En base a nuestras variables independientes y nuestra variable de respuesta, nuestro modelo logit se expresaria de la siguiente manera:

$$
\log\left(\frac{\pi}{1-\pi}\right) = \beta_0 + \beta_1 \cdot \text{Glucose} + \beta_2 \cdot \text{Insulin} + \beta_3 \cdot \text{BMI} + \beta_4 \cdot \text{Age} + \beta_5 \cdot \text{dpf}
$$

Por lo que utilizando la función `glm` para estimar el modelo logit de manera facil, los resultados serian los siguientes:

In [ ]:
modelo <- glm(diabetes ~ Glucose + Insulin + BMI + Age + DiabetesPedigreeFunction,
              data = df,
              family = binomial(link = "logit"))

summary(modelo)


Call:
glm(formula = diabetes ~ Glucose + Insulin + BMI + Age + DiabetesPedigreeFunction, 
    family = binomial(link = "logit"), data = df)

Coefficients:
                          Estimate Std. Error z value Pr(>|z|)    
(Intercept)              -9.405679   0.728933 -12.903  < 2e-16 ***
Glucose                   0.036522   0.003806   9.596  < 2e-16 ***
Insulin                  -0.001155   0.001143  -1.010 0.312310    
BMI                       0.090216   0.014592   6.183 6.31e-10 ***
Age                       0.029073   0.007746   3.753 0.000175 ***
DiabetesPedigreeFunction  0.819373   0.292122   2.805 0.005033 ** 
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 991.38  on 766  degrees of freedom
Residual deviance: 728.68  on 761  degrees of freedom
  (1 observation deleted due to missingness)
AIC: 740.68

Number of Fisher Scoring iterations: 5


### 1.3 Resultados del modelo
A partir de los resultados anteriores, obtenemos el modelo logístico dado como:

$$
logit(\pi)=\log\left(\frac{\pi}{1-\pi}\right) = -9.4056 + 0.0365\cdot \text{Glucose} + -0.0011 \cdot \text{Insulin} + 0.0902 \cdot \text{BMI} + 0.0290 \cdot \text{Age} + 0.8193 \cdot \text{dpf}
$$

## 1.4 Estimación del modelo **logit** paso a paso

### 1.4.1 Organización de los datos

El `dataframe` posee **datos individuales** que se acomodan en una matriz de diseño (con intercepto) $X$, una matriz de respuesta $\underline{y}$ y una matriz en donde se encuentran los datos individuales $\underline{n}$.

Por lo que la ecuacion del modelo logit de forma matricial se representa de la sig manera:

$$
\log\left(\frac{\pi}{1-\pi}\right) = X\beta
$$

donde:
- $X$ es la matriz de diseño
- $\beta$ es el vector de coeficientes
- $\pi$ es la probabilidad de que $Y$ $=$ $1$


In [ ]:
# Construyendo la matriz de diseño y vector de respuesta
X <- model.matrix(modelo)       # Con intercepto incluido
y <- modelo$y                   # Variable dependiente (0 o 1)
n <- rep(1, length(y))          # Cada individuo es un caso


### 1.4.2 Determinación de $\beta$'s mediante puntaje de Fisher

La expresión para determinar los coeficientes del modelo logit mediante el puntaje de fisher es la siguiente:

$$
\beta_{i+1}=\beta_i+(X^TWX)^{-1}X^T(\underline{y}-\underline{\mu})
$$
donde:  
- $\underline{\mu} = \underline{n}\cdot\underline{\pi}$
- $W$ = $diag(\pi_i(1−\pi_i))$
- $X$: matriz de diseño (con intercepto)
- $\underline{y}$: matriz de respuesta


In [ ]:

# Función de puntaje de Fisher
fisher_scoring_individual <- function(X, y, n, beta_init, tol = 1e-8, max_iter = 100) {
  beta <- beta_init
  iter <- 0
  change <- Inf

  while (change > tol && iter < max_iter) {
    iter <- iter + 1

    eta <- X %*% beta
    pi <- plogis(eta)                  # Probabilidad estimada

    W <- diag(as.vector(pi * (1 - pi)))  # Matriz de pesos (n = 1, así que se omite)
    mu <- pi                            # Media esperada

    #Matriz de informaciòn de Fisher
    info_fisher <- t(X) %*% W %*% X
    beta_new <- beta + solve(info_fisher) %*% t(X) %*% (y - mu)

    change <- sqrt(sum((beta_new - beta)^2))
    beta <- beta_new
  }

  if (iter == max_iter) warning("No convergió tras el máximo de iteraciones")

  cov_matrix <- solve(info_fisher)
  std_errors <- sqrt(diag(cov_matrix))
  z_values <- beta / std_errors
  p_values <- 2 * pnorm(-abs(z_values))

  # Resultado
  coefficients_df <- data.frame(
    Estimate = round(beta, 4),
    `Std. Error` = round(std_errors, 4),
    `z value` = round(z_values, 3),
    `Pr(>|z|)` = format.pval(p_values, digits = 3),
    check.names = FALSE
  )

  list(
    coefficients = coefficients_df,
    iterations = paste("Número de iteraciones:", iter)
  )
}


Al aplicar la función `fisher_scoring` anterior, obtenemos los $\beta$'s, sus `z_values` y `p_values` **son los mismos que se obtuvieron al utilizar la funcion `glm`**:

In [ ]:
# Llamamos a la función con valores iniciales
resultado <- fisher_scoring_individual(X, y, n, beta_init = rep(0, ncol(X)))

print(resultado$coefficients)
print(resultado$iterations)

                         Estimate Std. Error z value Pr(>|z|)
(Intercept)               -9.4057     0.7289 -12.903  < 2e-16
Glucose                    0.0365     0.0038   9.596  < 2e-16
Insulin                   -0.0012     0.0011  -1.010 0.312310
BMI                        0.0902     0.0146   6.183 6.31e-10
Age                        0.0291     0.0077   3.753 0.000175
DiabetesPedigreeFunction   0.8194     0.2921   2.805 0.005033
[1] "Número de iteraciones: 6"


In [ ]:
#Obtenemos los Odds Ratio con esta función
exp(coef(modelo))

(Intercept)                  Glucose                  Insulin 
            8.225557e-05             1.037197e+00             9.988455e-01 
                     BMI                      Age DiabetesPedigreeFunction 
            1.094410e+00             1.029500e+00             2.269076e+00

### 1.4.3 Interpretaciòn de los Odd - Ratio
Aplicando la exponencial a los coeficientes:

$O$$R$ $=$ $e$$^\beta$


- Glucose = $exp^{0.0365}\approx 1.037$ = 	Por cada 1 mg/dL de glucosa, el riesgo de diabetes **aumenta 3.7%**.
- Insulina = $exp^{-0.0012}\approx 0.998$ =
-  Índice de Masa Corporal (BMI) = $exp^{0.0902}\approx 1.094$ =  Por cada unidad de IMC incrementa el riesgo en **9.4%**.
-  Edad = $exp^{0.0291}\approx 1.029$ =  Por cada año adicional aumenta el riesgo en **2.9%**.
-  DiabetesPedigreeFunction (dpf) = $exp^{0.8194}\approx 2.269$ =  Quienes tienen mayor predisposición genética tienen **2.27 veces más riesgo**.


In [ ]:
#Obtenemos el pseudo R2 (McFadden) para evaluar la bondad de ajuste del modelo:
library(pscl)
pR2(modelo)

Classes and Methods for R originally developed in the
Political Science Computational Laboratory
Department of Political Science
Stanford University (2002-2015),
by and under the direction of Simon Jackman.
hurdle and zeroinfl functions by Achim Zeileis.



fitting null model for pseudo-r2


llh      llhNull           G2     McFadden         r2ML         r2CU 
-364.3417588 -495.6879356  262.6923537    0.2649776    0.2900021    0.3997676

### 1.4.4 Interpretaciòn de la bondad de ajuste:
Utilizaremos una medida similar dentro de nuestro modelo ya que el $R$$^2$ no existe dentro del modelo logit, por lue se estimo mediante la funciòn de arriba el **pseudo $R$$^2$**.

Donde:
- McFadden $R$$^2$ > 0.2 = bueno
- McFadden $R$$^2$ > 0.4 = excelente.

Segun los resultados obtenidos arriba, nuestro modelo cuenta con un McFadden R² de ~0.265, lo cual indica un buen ajuste.

Nota: Esta medida de ajuste se encuentra en Econometria, Gujarti(pag.563).

